# Configururation

### Enviornment Configuration

In [ ]:
ENV_USE_FRAME_STACK = False
ENV_FRAME_STACK_COUNT = 4
ENV_USE_GRAYSCALE_OBSERVATION = False

### Model Configuration

In [ ]:
MODEL_USE_CNN = True
MODEL_DEVICE = "auto" # "auto", "cuda", "cpu", or "mps"

### Traning Configuration

In [ ]:
TRANING_NAME = "baseline-CNN-MPS-4"
TRAINING_USE_PRIORITIZED_REPLAY = False
TRAINING_EVAL_FREQUENCY = 5000

# Setup Minedojo Environment

In [1]:
from Environments import SkyRunner, MultithreadGym

# Multithreaded environment wrapper
env = MultithreadGym.MultithreadGym(thread_int=1, env_int=1, frame_stack=ENV_USE_FRAME_STACK, frames_int=ENV_FRAME_STACK_COUNT, use_grayscale=ENV_USE_GRAYSCALE_OBSERVATION)

[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.


starting Reloader 0
ThreadID: 0 has received an enviornment from queue. Reset of environement is being prepeared


[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.
[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.


Unable to reset enviornment due to an exception
Traceback (most recent call last):
  File "/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/Environments/SkyRunner.py", line 83, in reset
    local_obs = self.env.reset()
  File "/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/Environments/Skyrunner_mission_interpreter.py", line 149, in reset
    self.env.reset()
  File "/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/venv/lib/python3.9/site-packages/gym/core.py", line 292, in reset
    return self.env.reset(**kwargs)
  File "/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/venv/lib/python3.9/site-packages/minedojo/sim/wrappers/ar_nn/delta_inventory_wrapper.py", line 78, in reset
    observation = self.env.reset(**kwargs)
  File "/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/venv/lib/python3.9/site-packages/gym/core.py", line 319, in reset
    observation = self.env.reset(**kwargs)
  File "/Users/kristian.aars/PycharmProjects/IDATT2502-Sky

-1

# Load EVAL-Enviornment

In [3]:
# Evaluation environment
eval_env = SkyRunner.CustomEnv(frame_stack=ENV_USE_FRAME_STACK, frames_int=ENV_FRAME_STACK_COUNT, use_grayscale=ENV_USE_GRAYSCALE_OBSERVATION)
eval_env.reset()

[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.


array([[[19, 21, 23, ..., 25, 23, 21],
        [21, 22, 25, ..., 27, 25, 23],
        [23, 24, 27, ..., 29, 27, 25],
        ...,
        [30, 36, 45, ..., 74, 63, 66],
        [33, 29, 37, ..., 71, 67, 64],
        [32, 31, 30, ..., 69, 65, 61]],

       [[23, 25, 27, ..., 30, 27, 25],
        [25, 26, 29, ..., 31, 29, 27],
        [26, 29, 31, ..., 34, 31, 29],
        ...,
        [40, 47, 58, ..., 47, 40, 42],
        [43, 38, 48, ..., 45, 43, 40],
        [41, 41, 39, ..., 44, 41, 39]],

       [[28, 31, 34, ..., 37, 34, 32],
        [31, 33, 36, ..., 39, 37, 34],
        [33, 36, 39, ..., 42, 40, 37],
        ...,
        [19, 22, 27, ..., 32, 28, 29],
        [20, 18, 23, ..., 31, 29, 28],
        [19, 19, 18, ..., 30, 28, 26]]], dtype=uint8)

# Begin training

In [8]:
import train_openAI
import importlib

importlib.reload(train_openAI)

train_openAI.train(env, eval_env=eval_env, eval_freq=TRAINING_EVAL_FREQUENCY, use_prioritized_replay=TRAINING_USE_PRIORITIZED_REPLAY, name=TRANING_NAME, device=MODEL_DEVICE, use_cnn=MODEL_USE_CNN)

Inventory and weather cleared!
Broken block detected. Moving to location 123
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Broken block detected. Moving to location 124
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Broken block detected. Moving to location 125
Inventory and weather cleared!
Broken block detected. Moving to location 126
Inventory and weather cleared!
Broken block detected. Moving to location 127
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Broken block detected. Moving to location 128
Inventory and weather cleared!
Inventory and weather cleared!
Inventory and weather cleared!
Broken block detected. Moving to location 129
Inventory and weather clear

# Shutdown Environments

In [13]:
env.close()
eval_env.close()

stopping Reloader 0


# Load existing model

In [ ]:
from stable_baselines3 import DQN
from CustomBaselines3.DoubleDQN import DoubleDQN

_env = env
model = DoubleDQN.load("/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/dDQN-checkpoints/checkpoint_30000_steps.zip")

obs = env.reset()
acc_r = 0
while True:
    act, st = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(act)

    acc_r += reward

    env.render()

    if done:
        obs = env.reset()
        print("Finished with reward %d" % acc_r)
        acc_r = 0